In [9]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
import re
from glove_routines import *
from text_classifier import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of text_classifier failed: Traceback (most recent call last):
  File "C:\Users\andre\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'KFold' is not defined
]


In [197]:

word = "lol wutwut (8 lol ? <user> lol , quoi ahah lol wutwut lol lol"
smiley = ":')"
print(re.escape(smiley))
test = word.split()
listesc = ".^$*+?{}[]\|()" #list of all escaped charactes
if('\(8' in listesc):
    print("loel")
pd = "\?" 
if re.search(r"(?:(?:(?:\b)|^)"+pd+"(?:(?=\b)|$)|(?:^|(?:\s))"+pd+"(?:$|(?=\s)))",word):
    count = re.findall(r"(?:(?:(?:\b)|^)"+pd+"(?:(?=\b)|$)|(?:^|(?:\s))"+pd+"(?:$|(?=\s)))",word)
    print(count)

\:\'\)
[' ?']


In [200]:
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')

    
    #count number of word/tweet and store it for both positive set and negative set
    word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
    for j in range(0,np.shape(pos_train)[0]):
        tweet = pos_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_pos[j] = size
    
    word_nbr_per_tweet_neg = np.zeros(np.shape(neg_train)[0])
    for j in range(0,np.shape(neg_train)[0]):
        tweet = neg_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_neg[j] = size
    
    print("counting ended")
    
    i=0
    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    vocab = open('data/vocab_cut.txt')
    #for each word, search if it is in pos_train or neg_train
    prevWord =""
    for word_ in vocab:
        word = word_.split("\n")[0]
        #if charac is ecaped, a special regex is used this is why there is a boolean
        if(re.escape(word) != word):
            word= re.escape(word)
        current_emb = embeddings[i]
        for j in range(0,np.shape(pos_train)[0]):
            #if yes, add its embeddings.
            #if word in pos_train[j]:
            if(prevWord != word):
                print(word)
                prevWord=word
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j])
                if(j < 20):
                    print(word,pos_train[j],len(count))
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        for j in range(0,np.shape(neg_train)[0]):
            #if word in neg_train[j]:
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j])
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        i+=1
        if(i%5000 ==0):
            print("5000 done")
    #then divide by number of words (averaging word vector over all words of the tweet)
    for i in range(0,np.shape(embeddings)[1]):
        training_set_pos[:,i+1] = training_set_pos[:,i+1]/word_nbr_per_tweet_pos
        training_set_neg[:,i+1] = training_set_neg[:,i+1]/word_nbr_per_tweet_neg


counting ended
\<user\>
\<user\> <user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
 1
\<user\> " <user> just put casper in a box ! " looved the battle ! #crakkbitch
 1
\<user\> <user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
 2
\<user\> <user> yay ! ! #lifecompleted . tweet / facebook me to let me know please
 1
\<user\> <user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : life , #yolo , becoming famous ? <3 14 #followmeplz ! <3 x15
 1
\<user\> workin hard or hardly workin rt <user> at hardee's with my future coworker <user>
 2
\<user\> <user> i saw . i'll be replying in a bit .
 1
\<user\> <user> anddd to cheer #nationals2013 ?
 1
\<user\> <user> agreed ! 12 more days left tho
 1
\<user\> like dammm <user> lexis u got a lot to say when ur on twitter lol
 1
\<user\> <user> at home affairs shall do it later
 1
\<user\> <user> a lot of parts of asia . es

KeyboardInterrupt: 

In [3]:
#!/usr/bin/env python3
from scipy.sparse import *
import csv
import numpy as np
import pandas as pd
import sklearn.linear_model as sk
import pickle
import random
import re

def construct_features():
    '''
    construct a feature representation of each training tweet 
    (by averaging the word vectors over all words of the tweet).
    '''
    #Load the training tweets and the built GloVe word embeddings.
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

    
    #count number of word/tweet and store it for both positive set and negative set
    word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
    for j in range(0,np.shape(pos_train)[0]):
        tweet = pos_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_pos[j] = size
    
    word_nbr_per_tweet_neg = np.zeros(np.shape(neg_train)[0])
    for j in range(0,np.shape(neg_train)[0]):
        tweet = neg_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_neg[j] = size
    
    print("counting ended")
    
    i=0
    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    vocab = open('data/vocab_cut.txt')
    #for each word, search if it is in pos_train or neg_train
    for j in range(0,np.shape(pos_train)[0]):
        list_word = pos_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
    for j in range(0,np.shape(neg_train)[0]):
        list_word = neg_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
    #then divide by number of words (averaging word vector over all words of the tweet)
    np.save('data/trainingset_pos', training_set_pos)
    np.save('data/trainingset_neg', training_set_neg)
    
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    #Construct the logistic regressor
    LR = sk.LogisticRegression()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    LR.fit(X,y)

    #Now we load and predict the data
    data = np.genfromtxt('data/test_data.txt', delimiter="\n",dtype=str)    
    idx = np.zeros(np.shape(data)[0])
    tweets = ["" for a in range(0,np.shape(data)[0])]
    for i in range(0,np.shape(data)[0]):
        spliter = data[i].split(",")
        idx[i] = spliter[0]
        tweet = spliter[1]
        for j in range(2,np.shape(spliter)[0]):
            tweet = tweet+","+spliter[j]
        tweets[i] = tweet
    #And now, predict the results
    topredict = construct_features_for_test_set(tweets)
    predictions = LR.predict(topredict)
    #Construct the submission
    predictions = predictions*2-1
    create_csv_submission(idx,predictions,"submission.csv")

def construct_features_for_test_set(test_set_tweet):
    embeddings = np.load('data/embeddings.npy')
    
    test_set = np.zeros(((np.shape(test_set_tweet)[0],np.shape(embeddings)[1])))
    #for each word, search if it is in a tweet
    i=0
    for word_ in vocab:
        word = word_.split("\n")[0]
        if(re.escape(word) != word):
            word= re.escape(word)
        current_emb = embeddings[i]
        for j in range(0,50):
            #if yes, add its embeddings.
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",test_set_tweet[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",test_set_tweet[j])
                test_set[j,:] += (len(count)*current_emb)
        i+=1
    #then divide by number of words (averaging word vector over all words of the tweet)
    for i in range(0,np.shape(embeddings)[1]):
        test_set[:,i] = test_set[:,i]/word_nbr_per_tweet
    return test_set
predict_labels()




In [13]:
embeddings = np.load('data/embeddings.npy')
with open('data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
    vocab_size = len(vocab)
print(vocab.get("<user>",-1))
print(embeddings[0])

0
[-0.26831595 -0.03828992 -0.23159524 -0.11406195 -0.09299987 -0.25714371
  0.00553967 -0.13492522 -0.35268678  0.04298145 -0.43047919 -0.13801229
  0.72375534  0.03839596 -0.20425634  0.16527752 -0.61413165  0.04258187
  0.09463027 -0.10918866]


In [23]:
def construct_features():
    '''
    construct a feature representation of each training tweet 
    (by averaging the word vectors over all words of the tweet).
    '''
    #Load the training tweets and the built GloVe word embeddings.
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    #for each word, search if it is in pos_train or neg_train
    for j in range(0,np.shape(pos_train)[0]):
        list_word = pos_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
        training_set_pos[j,:] = training_set_pos[j,:]/len(list_word)
    for j in range(0,np.shape(neg_train)[0]):
        list_word = neg_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
        training_set_neg[j,:] = training_set_neg[j,:]/len(list_word)
    np.save('data/trainingset_pos', training_set_pos)
    np.save('data/trainingset_neg', training_set_neg)

In [38]:
def construct_features():
    '''
    construct a feature representation of each training tweet 
    (by averaging the word vectors over all words of the tweet).
    '''
    #Load the training tweets and the built GloVe word embeddings.
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    #for each word, search if it is in pos_train or neg_train
    for j in range(0,np.shape(pos_train)[0]):
        list_word = pos_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
    for j in range(0,np.shape(neg_train)[0]):
        list_word = neg_train[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
    np.save('data/trainingset_pos', training_set_pos)
    np.save('data/trainingset_neg', training_set_neg)
    
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    #Construct the logistic regressor
    LR = sk.LogisticRegression()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    LR.fit(X,y)

    #Now we load and predict the data
    data = np.genfromtxt('data/test_data.txt', delimiter="\n",dtype=str)    
    idx = np.zeros(np.shape(data)[0])
    tweets = ["" for a in range(0,np.shape(data)[0])]
    for i in range(0,np.shape(data)[0]):
        spliter = data[i].split(",")
        idx[i] = spliter[0]
        tweet = spliter[1]
        for j in range(2,np.shape(spliter)[0]):
            tweet = tweet+","+spliter[j]
        tweets[i] = tweet
    #And now, predict the results
    topredict = construct_features_for_test_set(tweets)
    predictions = LR.predict(topredict)
    #Construct the submission
    predictions = predictions*2-1
    create_csv_submission(idx,predictions,"submission.csv")

def construct_features_for_test_set(test_set_tweet):
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    
    test_set = np.zeros((np.shape(test_set_tweet)[0],np.shape(embeddings)[1]))
    #for each word, search if it is in a tweet
    for j in range(0,np.shape(test_set)[0]):
        list_word = test_set_tweet[j].split()
        for i in list_word:
            idx = vocab.get(i,-1)
            if(idx>=0):
                test_set[j,:] += embeddings[idx]
    #then divide by number of words (averaging word vector over all words of the tweet)
    return test_set